<a href="https://colab.research.google.com/github/LRLeite/Data-Analytics/blob/main/Projeto%20IMDb/IMDb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **🎬 Análise das Avaliações de Filmes e Séries, Segundo o IMDb**


Este notebook foi desenvolvido para realizar uma análise sobre os filmes e séries avaliados no IMDb. O objetivo central é analisar as avaliações do público sobre filmes e séries.<br>



## **🎯 Objetivos da Análise**

Ao longo deste notebook, as seguintes perguntas foram abordadas e respondidas:

* Qual a quantidade de títulos por categoria (Filmes longa metragem e Séries de TV)?

* Quais são os TOP 10 Filmes longa metragem com as melhores avaliações?

* Quais são os TOP 10 Filmes longa metragem com as piores avaliações?

* Quais são as TOP 10 Séries de TV com as melhores avaliações?

* Quais são as TOP 10 Séries de TV com as piores avaliações?

* Quais filmes possuem a maior duração?
<br><br>

**Fonte dos dados:** [IMDB](https://datasets.imdbws.com/)<br>
**Informações sobre os dados:** https://www.imdb.com/interfaces/

# **Bibliotecas**

In [1]:
import pandas as pd
import gzip
import numpy as np

# **Importar os dados**

In [2]:
#Salvar o link dos dados das informações básicas num objeto
url_basics = 'https://datasets.imdbws.com/title.basics.tsv.gz'

#Importar o dataset com os dados das informações básicas
basics = pd.read_csv(url_basics, compression = 'gzip', sep = '\t', usecols = ['tconst','titleType', 'originalTitle', 'startYear', 'endYear', 'runtimeMinutes', 'genres'])
basics.head()

,tconst,titleType,originalTitle,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,1892,\N,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,1893,\N,1,Short


In [3]:
#Verificar estrutura dos dados
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11734860 entries, 0 to 11734859
Data columns (total 7 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   originalTitle   object
 3   startYear       object
 4   endYear         object
 5   runtimeMinutes  object
 6   genres          object
dtypes: object(7)
memory usage: 626.7+ MB


In [4]:
#Salvar o link dos dados das avaliações num objeto
url_ratings = 'https://datasets.imdbws.com/title.ratings.tsv.gz'

#Importar o dataset com os dados das avaliações
ratings = pd.read_csv(url_ratings, compression = 'gzip', sep = '\t')
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2164
1,tt0000002,5.5,296
2,tt0000003,6.5,2216
3,tt0000004,5.3,189
4,tt0000005,6.2,2955


In [5]:
#Verificar estrutura dos dados
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1581386 entries, 0 to 1581385
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1581386 non-null  object 
 1   averageRating  1581386 non-null  float64
 2   numVotes       1581386 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 36.2+ MB


# **Preparar os datasets**

In [6]:
#Verificar a quantidade de valores únicos em ratings
ratings.tconst.nunique()

1581386

In [7]:
#Verificar a quantidade de títulos únicos em basics
basics.originalTitle.nunique()

5310911

In [8]:
#Verificar se existem 'tconst' duplicados em basics
basics[basics['tconst'].duplicated(keep = False)]

,tconst,titleType,originalTitle,startYear,endYear,runtimeMinutes,genres


In [9]:
#Verificar se existem ´tconst´ duplicados em ratings
ratings[ratings['tconst'].duplicated(keep = False)]

,tconst,averageRating,numVotes


Isso mostra que nem todos os títulos possuem avaliação (existem 1.580.294 avaliações contra 5.308.082 de títulos).<br>
Para os propósitos desta análise, os títulos e as avaliações serão utilizados. Dessa forma, ao juntar os datasets utilizarei o inner join para ficar apenas com as observações que possuem títulos e avaliações.

## **Realizar join das tabelas para visualizar as avaliações de cada título**

In [10]:
#Verificar os tipos de conteúdos no dataset, para selecionar apenas filmes e séries para análise
basics.titleType.unique()

array(['short', 'movie', 'tvShort', 'tvMovie', 'tvEpisode', 'tvSeries',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame', 'tvPilot'],
      dtype=object)

In [11]:
#Join das tabelas, incluindo "title", "type", "year", "runtime" dos filmes longa mentragem, filmes de tc e séries de TV com suas respectivas avaliações
#Para selecionar apenas os filmes longa metragem, o tipo de titulo deve ser 'movie'
df_merge = pd.merge(basics[['tconst', 'titleType', 'originalTitle', 'runtimeMinutes']].loc[basics.titleType.isin(['movie', 'tvSeries'])], ratings[['tconst','averageRating', 'numVotes']], on = 'tconst', how = 'inner')
df_merge.head()

,tconst,titleType,originalTitle,runtimeMinutes,averageRating,numVotes
0,tt0000009,movie,Miss Jerry,45,5.4,226
1,tt0000147,movie,The Corbett-Fitzsimmons Fight,100,5.3,562
2,tt0000502,movie,Bohemios,100,3.6,22
3,tt0000574,movie,The Story of the Kelly Gang,70,6.0,1003
4,tt0000591,movie,L'enfant prodigue,90,5.4,33


In [12]:
#Verificar valores ausentes
df_merge.isna().sum()

,0
tconst,0
titleType,0
originalTitle,1
runtimeMinutes,0
averageRating,0
numVotes,0


In [13]:
#Verificar NA
df_merge[df_merge['originalTitle'].isna()]

,tconst,titleType,originalTitle,runtimeMinutes,averageRating,numVotes
388525,tt5774076,movie,NaN,\N,7.1,10


In [14]:
#Como o título será utilizado nas análises e essa observação não possui, irei remover do dataset
df_merge = df_merge.dropna(subset = ['originalTitle'])

In [15]:
#Verificar a estrutura dos dados
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Index: 436176 entries, 0 to 436176
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          436176 non-null  object 
 1   titleType       436176 non-null  object 
 2   originalTitle   436176 non-null  object 
 3   runtimeMinutes  436176 non-null  object 
 4   averageRating   436176 non-null  float64
 5   numVotes        436176 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 23.3+ MB




Conforme informado no site do [IMDb](https:www.imdb.com/interfaces/), '\\\N' é usado para indicar que um determinado campo está ausente ou nulo para esse título/nome". Além disso, as colunas 'startYear', 'endYear', 'runtimeMinutes' são do tipo 'object', o que impossibilita análises númericas que serão realizadas na sequência. Dessa forma, para converter as colunas para tipo númerico na sequência, substituirei os valores '\\\N ' por nan, para evitar a distorção nas avaliações e enviesamento dos rankings.



In [16]:
#Verificar se existe '\N' em alguma coluna do dataset e contar
total_n_count = 0
for col in df_merge.columns:
    #Converte a cluna para string antes de checar
    col_as_str = df_merge[col].astype(str)
    n_count = (col_as_str == '\\N').sum()
    if n_count > 0:
         print(f"A coluna '{col}' contém {n_count} valores exatamente iguais a '\\N'")
         total_n_count += n_count
    else:
        print(f"Não tem nenhum valor igual a '\\N' em {col}")

Não tem nenhum valor igual a '\N' em tconst
Não tem nenhum valor igual a '\N' em titleType
Não tem nenhum valor igual a '\N' em originalTitle
A coluna 'runtimeMinutes' contém 82188 valores exatamente iguais a '\N'
Não tem nenhum valor igual a '\N' em averageRating
Não tem nenhum valor igual a '\N' em numVotes


In [17]:
#Substituir '\\N' por NaN
df_merge = df_merge.replace('\\N', np.nan)

In [18]:
#Converter colunas para tipo Int64
df_merge[['runtimeMinutes', 'numVotes']] = df_merge[['runtimeMinutes', 'numVotes']].astype('Int64')
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Index: 436176 entries, 0 to 436176
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          436176 non-null  object 
 1   titleType       436176 non-null  object 
 2   originalTitle   436176 non-null  object 
 3   runtimeMinutes  353988 non-null  Int64  
 4   averageRating   436176 non-null  float64
 5   numVotes        436176 non-null  Int64  
dtypes: Int64(2), float64(1), object(3)
memory usage: 24.1+ MB


In [19]:
df_merge.isna().sum()

,0
tconst,0
titleType,0
originalTitle,0
runtimeMinutes,82188
averageRating,0
numVotes,0




# **Análises**



##**Disclaimer**

O IMDb avalia filmes e séries usando um sistema complexo, não apenas uma média simples de votos. Os principais critérios são:

* Classificação Ponderada: O IMDb usa uma média ponderada para as classificações, onde nem todos os votos têm o mesmo impacto. O método exato é mantido em sigilo para evitar manipulações

* Votos para o Top 250: são considerados apenas os votos de "eleitores regulares", cujos critérios são secretos. Um título precisa de no mínimo 25.000 votos para ser considerado, e a média geral de votos do banco de dados (aproximadamente 7.0) também influencia.

* Avaliações de Usuários: usuários podem enviar avaliações detalhadas, seguindo diretrizes específicas, como avisar sobre spoilers e focar no conteúdo.

Dessa forma, para os fins desta análise, utilizarei o único critério concreto (>= 25.000 votos) e isso pode refletir em títulos diferentes do ranking oficial do IMDb.

## **Quantidade de títulos por categoria**

In [20]:
df_merge[['originalTitle', 'titleType']].groupby('titleType').count()

,originalTitle
titleType,
movie,331476
tvSeries,104700


## **TOP 10 Filmes longa metragem com as melhores avaliações**

In [21]:
df_merge[['originalTitle', 'averageRating', 'numVotes']].loc[(df_merge.titleType == 'movie') & (df_merge.numVotes >= 25000)].sort_values(by = 'averageRating', ascending = False).head(10)

,originalTitle,averageRating,numVotes
67084,The Shawshank Redemption,9.3,3059168
113289,Hababam Sinifi,9.2,44623
39113,The Godfather,9.2,2135475
24964,12 Angry Men,9.0,932462
65248,Schindler's List,9.0,1530613
162025,The Dark Knight,9.0,3033780
41220,The Godfather Part II,9.0,1436782
87208,The Lord of the Rings: The Return of the King,9.0,2086829
72511,The Lord of the Rings: The Fellowship of the Ring,8.9,2117394
113655,Tosun Pasa,8.9,25619


## **TOP 10 Filmes longa metragem com as piores avaliações**


In [22]:
df_merge[['originalTitle', 'averageRating', 'numVotes']].loc[(df_merge.titleType == 'movie') & (df_merge.numVotes >= 25000 )].sort_values(by = 'averageRating').head(10)

,originalTitle,averageRating,numVotes
408693,Cumali Ceber,1.0,39591
391892,Reis,1.0,74359
392622,Smolensk,1.2,40356
416596,Sadak 2,1.2,96987
237062,Elk*rtuk,1.5,40839
118768,Superbabies: Baby Geniuses 2,1.5,32014
258502,Justin Bieber: Never Say Never,1.7,76516
217497,Birdemic: Shock and Terror,1.7,26388
33283,Manos: The Hands of Fate,1.7,38151
368589,Kod Adi K.O.Z.,1.7,27604


## **TOP 10 Séries de TV com as melhores avaliações**

In [23]:
df_merge[['originalTitle', 'averageRating', 'numVotes']].loc[(df_merge.titleType == 'tvSeries') & (df_merge.numVotes >= 25000 )].sort_values(by = 'averageRating', ascending = False).head(10)

,originalTitle,averageRating,numVotes
174892,Breaking Bad,9.5,2347081
414252,Bluey,9.3,39260
152902,Avatar: The Last Airbender,9.3,408912
129082,The Wire,9.3,405603
328824,Sapne Vs Everyone,9.3,74018
233047,Aspirants,9.2,315910
430858,Our Planet,9.2,57229
176211,Game of Thrones,9.2,2445625
79928,The Sopranos,9.2,529611
40864,The World at War,9.2,33292


## **TOP 10 Séries de TV com as piores avaliações**

In [24]:
df_merge[['originalTitle', 'averageRating', 'numVotes']].loc[(df_merge.titleType == 'tvSeries') & (df_merge.numVotes >= 25000 )].sort_values(by = 'averageRating').head(10)

,originalTitle,averageRating,numVotes
230206,Velma,1.6,80632
189917,Keeping Up with the Kardashians,2.9,31990
425270,Batwoman,3.6,47527
434048,Resident Evil,4.2,47966
206838,The Acolyte,4.2,135510
239567,The Idol,4.4,114631
193113,Tandav,4.7,61552
363867,Inhumans,4.9,30043
274329,Indian Police Force,5.0,58569
234201,Raketsonyeondan,5.0,29521


## **Filmes com maior duração**

In [25]:
df_merge[['originalTitle', 'runtimeMinutes']].loc[df_merge.titleType == 'movie'].sort_values(by = 'runtimeMinutes', ascending = False).head(10)

,originalTitle,runtimeMinutes
324334,100,59460
420874,Logistics,51420
359069,Ambiancé,43200
309623,Modern Times Forever,14400
189638,Qw,10062
298205,Beijing 2003,9000
379008,London EC1,5460
122981,The Cure for Insomnia,5220
137345,The Longest Most Meaningless Movie in the World,2880
191271,Five-Year Diary,2160
